Works with `d05199b`

In [1]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import os
from glob import glob

In [2]:
from allensdk.brain_observatory.sync_dataset import Dataset
from allensdk.brain_observatory.ecephys.align_timestamps.barcode_sync_dataset import BarcodeSyncDataset
from allensdk.brain_observatory.ecephys.align_timestamps.channel_states import extract_barcodes_from_states, extract_splits_from_states
from allensdk.brain_observatory.ecephys.align_timestamps.barcode import match_barcodes

In [3]:
from tbd_eeg.data_analysis.eegutils import EEGexp

In [4]:
%matplotlib widget

### Load experiment

In [26]:
data_folder = r"/allen/programs/braintv/workgroups/nc-ophys/Leslie/eeg_pilot/mouse496220/pilot1_2020.01.10/recording1/"
pilotexp = EEGexp(data_folder)

In [27]:
# memmap raw data to get size
datamm = pilotexp.memmap_EEGdata()
samp_rate = pilotexp.sample_rate
print(datamm.shape)

Loading /allen/programs/braintv/workgroups/nc-ophys/Leslie/eeg_pilot/mouse496220/pilot1_2020.01.10/recording1/continuous/Rhythm_FPGA-111.0/continuous.dat
(6987264, 32)


#### Get barcodes from sync file

In [28]:
print(pilotexp.sync_file)

/allen/programs/braintv/workgroups/nc-ophys/Leslie/eeg_pilot/mouse496220/pilot1_2020.01.10/recording1/mouse496220_200110_154654.sync


In [29]:
# extract sync data
sync_dataset = BarcodeSyncDataset.factory(pilotexp.sync_file)
sync_times, sync_codes = sync_dataset.extract_barcodes()

In [30]:
print('Sync times:')
print(sync_times[0:5])
print(np.shape(sync_times))
print('Sync barcodes:')
print(sync_codes[0:5])
print(np.shape(sync_codes))

Sync times:
[ 32.01896  62.95234  93.88577 124.81918 155.75258]
(92,)
Sync barcodes:
[728806759.0, 176515105.0, 943159851.0, 214275341.0, 142382483.0]
(92,)


#### Get barcodes from EEG data stream

In [31]:
channel_states = np.load(os.path.join(pilotexp.data_folder, 'events', 'Rhythm_FPGA-111.0', 'TTL_1', 'channel_states.npy'))
timestamps = np.load(os.path.join(pilotexp.data_folder, 'events', 'Rhythm_FPGA-111.0', 'TTL_1', 'timestamps.npy'))

In [32]:
print(timestamps[0:5])

[1882076 1882126 1882176 1882466 1882539]


##### Right now, the channel_states file reads -8, +8, -8...due to the barcodes being read in on digital line 8. Going to try dividing to make it 1s, because that is what the barcodes functions look for.

In [33]:
channel_states = channel_states // 8

In [34]:
eeg_barcode_times, eeg_barcodes = extract_barcodes_from_states(channel_states, timestamps, samp_rate)
eeg_split_times = extract_splits_from_states(channel_states, timestamps, samp_rate)

In [35]:
print('EEG barcode times:')
print(eeg_barcode_times[0:5])
print(np.shape(eeg_barcode_times))
print('EEG_barcodes:')
print(eeg_barcodes[0:5])
print(np.shape(eeg_barcodes))
print(eeg_split_times)

EEG barcode times:
[783.7644 814.698  845.632  876.5656 907.4996]
(89,)
EEG_barcodes:
[943159851.0, 214275341.0, 142382483.0, 459255029.0, 821289793.0]
(89,)
[0]


In [36]:
for idx, split_time in enumerate(eeg_split_times):
    min_time = eeg_split_times[idx]

    if idx == (len(eeg_split_times) - 1):
        max_time = np.Inf
    else:
        max_time = eeg_split_times[idx+1]
    print('Split %d: begin %f , end %f' % (idx, min_time, max_time))

Split 0: begin 0.000000 , end inf


#### Need to take the timestamps from every barcode and find difference between each one. Is this diff constant for both?

In [18]:
fig=plt.figure(figsize=(6,4))
subfignum = len(eeg_split_times)
for idx, split_time in enumerate(eeg_split_times):
    min_time = eeg_split_times[idx]

    if idx == (len(eeg_split_times) - 1):
        max_time = np.Inf
    else:
        max_time = eeg_split_times[idx+1]
        
    split_barcodes = np.array(eeg_barcodes)[np.where((eeg_barcode_times > min_time) * (eeg_barcode_times < max_time))[0]]
    syncinds = np.isin(sync_codes, split_barcodes)
    eeginds = np.isin(eeg_barcodes, split_barcodes)
    overlapping_sync_times = sync_times[syncinds]
    overlapping_eeg_times = eeg_barcode_times[eeginds]
    
    ax=plt.subplot(1, subfignum, idx+1)
    ax.plot(overlapping_eeg_times, overlapping_sync_times - overlapping_eeg_times, '.')
    ax.set_title('Split #%d' % (idx+1))

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: operands could not be broadcast together with shapes (87,) (89,) 

### Need to align timestamps between each barcode.

In [22]:
origTS = np.load(os.path.join(pilotexp.data_folder, 'continuous', 'Rhythm_FPGA-111.0', 'timestamps.npy'))
print(origTS.shape)

(6987264,)


In [23]:
print(origTS[-1])
print(origTS[0:5])

8818175
[1830912 1830913 1830914 1830915 1830916]


##### Note: EEG timestamps do not begin at 0; they begin when the recording was started relative to starting OpenEphys.

### Need to loop through the eeg_split_times to align each section separately.
##### There shouldn't be any split times for the EEG recordings, but maybe safest to keep this in.

In [24]:
### Made necessary changes to accommodate this difference (from Npx recs), need to double-
### check that it still works if there is a split recording******

aligned_timestamps = []
for idx, split_time in enumerate(eeg_split_times):
    if idx == 0: # in case the EEG timestamps do not begin at 0
        min_sample = origTS[idx]
        min_time = min_sample/samp_rate
    else:
        min_time = eeg_split_times[idx]
        min_sample = int(min_time*samp_rate)

    if idx == (len(eeg_split_times) - 1):
        max_sample = origTS[-1] # in case the EEG timestamps do not begin at 0
        max_time = max_sample/samp_rate
    else:
        max_time = eeg_split_times[idx+1]
        max_sample = int(max_time*samp_rate)
        
    # this finds the barcodes from this split that are common to the sync_ and eeg_barcodes
    split_barcodes = np.array(eeg_barcodes)[np.where((eeg_barcode_times > min_time) * (eeg_barcode_times < max_time))[0]]
    syncinds = np.isin(sync_codes, split_barcodes)
    eeginds = np.isin(eeg_barcodes, split_barcodes)
    overlapping_sync_times = sync_times[syncinds]
    overlapping_eeg_times = eeg_barcode_times[eeginds]

    barcode_inds = (overlapping_eeg_times*samp_rate).astype('int64')
    xts = np.arange(barcode_inds[0], barcode_inds[-1]+1, 1)
    midts = np.interp(xts, barcode_inds, overlapping_sync_times)

    # fit a line to first/last 10 barcodes; this will error if # barcodes in split is less than 10
    n_fit = 2
    slope1, intercept1, r1, p1, stderr1 = st.linregress(overlapping_eeg_times[0:n_fit], overlapping_sync_times[0:n_fit])
    begts = np.arange(min_sample, barcode_inds[0])*(slope1/samp_rate) + intercept1

    slope2, intercept2, r2, p2, stderr2 = st.linregress(overlapping_eeg_times[-n_fit:], overlapping_sync_times[-n_fit:])
    endts = np.arange(barcode_inds[-1]+1, max_sample+1, 1)*(slope2/samp_rate) + intercept2

    aligned_timestamps = np.concatenate((aligned_timestamps, begts, midts, endts))

ValueError: fp and xp are not of the same length.

In [ ]:
print(aligned_timestamps[0:10])

#### Sanity check

In [19]:
if len(aligned_timestamps) == len(origTS):
    print('Wahooo, same length!')
else:
    print('Uh oh...?')

NameError: name 'aligned_timestamps' is not defined

In [20]:
if (aligned_timestamps > 9000).sum() == 0:
    print('All looks good, no crazy high timestamps')
else:
    print((aligned_timestamps > 9000).sum())

NameError: name 'aligned_timestamps' is not defined

In [21]:
if np.all(aligned_timestamps[1:] >= aligned_timestamps[:-1], axis=0): # should be TRUE!!! for only-increasing arrays (like timestamps should be)
    print('Good to go, timestamps are only increasing')

NameError: name 'aligned_timestamps' is not defined

### Check offsets now

In [34]:
fig=plt.figure(figsize=(6,4))
subfignum = len(eeg_split_times)
for idx, split_time in enumerate(eeg_split_times):
    min_time = eeg_split_times[idx]

    if idx == (len(eeg_split_times) - 1):
        max_time = np.Inf
    else:
        max_time = eeg_split_times[idx+1]
        
    split_barcodes = np.array(eeg_barcodes)[np.where((eeg_barcode_times > min_time) * (eeg_barcode_times < max_time))[0]]
    syncinds = np.isin(sync_codes, split_barcodes)
    eeginds = np.isin(eeg_barcodes, split_barcodes)
    overlapping_sync_times = sync_times[syncinds]
    overlapping_eeg_times = eeg_barcode_times[eeginds]
    new_eeg_times = aligned_timestamps[(overlapping_eeg_times*samp_rate).astype('int64') - origTS[0]]
    # something is wrong with above line...
    
    ax=plt.subplot(1,subfignum,idx+1)
    ax.plot(new_eeg_times,overlapping_sync_times-new_eeg_times,'.')
    ax.set_title('Split #%d' % (idx+1))

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### All looks good, now save.

In [35]:
np.save(os.path.join(pilotexp.data_folder, 'continuous', 'Rhythm_FPGA-111.0', 'timestamps_master_clock.npy'), aligned_timestamps, allow_pickle=False)